# Project - 1 : Apache Spark 

Instructor: Ramesh Yerraballi
TA: Madhumitha Sakthi
Semester: Fall 2019
Due Date: 11:59pm, Monday 9/16

This project is based on Map-Reduce Framework. In these you will get to work with Spark and will get to know how 
does spark work, what functionalities does spark provide, what does map-reduce framework do and why is it useful. 

In this project you will be implementing a basic song recommender system. You will be given a dataset where there are multiple csv files. These csv files have data corresponding to song play count and song information.

The data you would be using will be provided in a zip file along with this notebook. The __msd.zip__ archive contains '_kaggle_visible_evaluation_triplets.txt_'. We will be using the visible part of the testing data to understand the working on Apache Spark.  The user's listening history is provided as: (user, song, play count). In another file, '_kaggle_songs.txt_', each song is marked using an index for easier representation of songs. 



What to turn in:
A zip folder which will have:
- Jupyter Notebook
- A brief report on what features you used for recommendation. And a brief explanation of flow of your code. For example,  what RDD does what or, why it was created.
- datasets folder with the csv files you are using in your notebook.
- Notebook should use relative path to the csv files in datasets folder.
- Name of the zip folder - <your\_name>\_<your\_partner_name>.zip


This project consists of 4 questions. 
1. Create an RDD with _msd_evalutation_triplets.txt_ and replace the song name with the song index from _msd_songs.txt_. Identify the number of songs that do not have any rating. 
2. Generate song ratings based on the song play count as a normalized score between 0 and 1. 
3. Identify the popular song based on this rating and recommend songs to user, given user id based on the algorithm used in Movie recommender system from class. 
4. Using Cosine similarity function, identify pair-wise similarity between each pair of users and generate the top 5 most similar users without an overlap in users. 

The above list is the higer level idea about the questions. 

In [2]:
### Starter code ####

##### These lines are to tell jupyter where to find Apache Spark ####
import findspark
# findspark.init('C:\\apachespark')
from pyspark import SparkConf, SparkContext
from itertools import accumulate
conf = SparkConf().setMaster("local[*]").setAppName("Songs")
# There are two configurable parameters
# 1. A cluster URL, namely  local  in this example, which tells Spark how to connect
# to a cluster.  local  is a special value that runs Spark on one thread on the local
# machine, without connecting to a cluster.
# 2. An application name, namely  Movies  in this example. This will identify your
# application on the cluster manager’s UI if you connect to a cluster.
sc = SparkContext(conf = conf)
##### These lines are to tell jupyter where to find Apache Spark ####

In [3]:
## Read triplet file into RDD
triplet_rdd = sc.textFile(r"msd/kaggle_visible_evaluation_triplets.txt") \
    .map(lambda line: line.split("\t")) 

## Read song file into RDD
songs_rdd = sc.textFile(r"msd/kaggle_songs.txt") \
    .map(lambda line: line.split(" ")) 

## Step 1: 
Replace song name with song index and identify the number of songs without user history

In [4]:
hacked_triplets = triplet_rdd.map(lambda x: (1,x))

In [5]:
hacked_songs = songs_rdd.map(lambda x: (1,x))

In [6]:
hacked_triplets.take(20)

[(1, ['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', 'SOBONKR12A58A7A7E0', '1']),
 (1, ['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', 'SOEGIYH12A6D4FC0E3', '1']),
 (1, ['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', 'SOFLJQZ12A6D4FADA6', '1']),
 (1, ['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', 'SOHTKMO12AB01843B0', '1']),
 (1, ['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', 'SODQZCY12A6D4F9D11', '1']),
 (1, ['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', 'SOXLOQG12AF72A2D55', '1']),
 (1, ['d7083f5e1d50c264277d624340edaaf3dc16095b', 'SOUVUHC12A67020E3B', '1']),
 (1, ['d7083f5e1d50c264277d624340edaaf3dc16095b', 'SOUQERE12A58A75633', '1']),
 (1, ['d7083f5e1d50c264277d624340edaaf3dc16095b', 'SOIPJAX12A8C141A2D', '1']),
 (1, ['d7083f5e1d50c264277d624340edaaf3dc16095b', 'SOEFCDJ12AB0185FA0', '2']),
 (1, ['d7083f5e1d50c264277d624340edaaf3dc16095b', 'SOATCSU12A8C13393A', '1']),
 (1, ['d7083f5e1d50c264277d624340edaaf3dc16095b', 'SOZPZGN12A8C135B45', '1']),
 (1, ['d7083f5e1d50c264277d624340edaaf3dc16095b', 'S

In [7]:
songs_rdd.take(10)

[['SOAAADD12AB018A9DD', '1'],
 ['SOAAADE12A6D4F80CC', '2'],
 ['SOAAADF12A8C13DF62', '3'],
 ['SOAAADZ12A8C1334FB', '4'],
 ['SOAAAFI12A6D4F9C66', '5'],
 ['SOAAAGK12AB0189572', '6'],
 ['SOAAAGN12AB017D672', '7'],
 ['SOAAAGO12A67AE0A0E', '8'],
 ['SOAAAGP12A6D4F7D1C', '9'],
 ['SOAAAGQ12A8C1420C8', '10']]

In [8]:
def matchSongName(row):
    if row[1] == row[2]:
        return True
    return False

In [9]:
# RDD With Song Index instead of Song Name
joined = hacked_triplets.join(hacked_songs) \
            .map(lambda x: [x[1][0][0] , x[1][0][1] , x[1][1][0] , x[1][1][1] , x[1][0][2]]) \
            .filter(matchSongName) \
            .map(lambda x: [x[0] , x[3] , x[4]])

joined.take(10)

[['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', '25150', '1'],
 ['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', '68212', '1'],
 ['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', '87433', '1'],
 ['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', '123630', '1'],
 ['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', '58821', '1'],
 ['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', '351764', '1'],
 ['d7083f5e1d50c264277d624340edaaf3dc16095b', '315822', '1'],
 ['d7083f5e1d50c264277d624340edaaf3dc16095b', '312787', '1'],
 ['d7083f5e1d50c264277d624340edaaf3dc16095b', '136848', '1'],
 ['d7083f5e1d50c264277d624340edaaf3dc16095b', '67457', '2']]

In [10]:
onlySongNames = triplet_rdd.map(lambda x: x[1]).collect()

In [11]:
song_set = set(onlySongNames)

In [12]:
# Number of missing songs
distinctSongs = songs_rdd.keys().collect()
missing_songs = len(distinctSongs) - len(song_set)
print(str(missing_songs) + " songs do not have a user history.")

223007 songs do not have a user history.


## Step 2:
Generate song ratings based on the play_count. For example, if (song_1, 5; song_2, 10; song_3, 5) i.e., song_1 is played 5 times, song_2 is played 10 times and song_3 is played 5 times, the normalized rating score should be 0.25, 0.5 and 0.25 respectively. 
Similarly, generate the rating for all the songs. You may notice that based on all songs, the rating is almost always very low. So, think of the best way to convert song count to ratings. (Hint: Try generating ratings based on each user's song play history)

In [13]:
song_counts_all = triplet_rdd.map(lambda x: [x[1], int(x[2])])

In [41]:
# Each Song with it's play count summed across the dataset
songs_count_unique = song_counts_all.reduceByKey(lambda x,y:x+y)
songs_count_unique.take(5)

[('SOBONKR12A58A7A7E0', 35432),
 ('SOFLJQZ12A6D4FADA6', 7895),
 ('SOHTKMO12AB01843B0', 10515),
 ('SOXLOQG12AF72A2D55', 4671),
 ('SOZPZGN12A8C135B45', 39)]

In [36]:
# Sum of all play counts
songs_count_sum=song_counts_all.map(lambda x:x[1]).sum()
songs_count_sum

4624340

In [40]:
#Maximum normalized score
songs_count_unique.map(lambda x: (x[0],x[1]/songs_count_sum)).map(lambda x: x[1]).max()

0.007662066370552338

In [16]:
sorted_songs = songs_count_unique.sortBy(lambda x: -x[1])
sorted_songs.take(10)

[('SOBONKR12A58A7A7E0', 35432),
 ('SOAUWYT12A81C206F1', 33179),
 ('SOSXLTC12AF72A7F54', 24359),
 ('SOFRQTD12A81C233C0', 19454),
 ('SOEGIYH12A6D4FC0E3', 17115),
 ('SOAXGDH12A8C13F8A1', 14279),
 ('SONYKOW12AB01849C9', 12392),
 ('SOVDSJC12A58A7A271', 11610),
 ('SOUFTBI12AB0183F65', 10794),
 ('SOHTKMO12AB01843B0', 10515)]

In [43]:
maxPlayed=int(sorted_songs.take(1)[0][1])
def rateSong(row):
    rating = round((10/maxPlayed)*row[1],2)
    rate = [row[0], rating]
    return rate

In [44]:
rated_songs = songs_count_unique.map(rateSong)

In [45]:
# Songs rated on the number of times it was played by any user - Highest played song=10, Songs played once is lowest~0
rated_songs.sortBy(lambda x:-x[1]).take(10)
#Top ten songs with ratings

[['SOBONKR12A58A7A7E0', 10.0],
 ['SOAUWYT12A81C206F1', 9.36],
 ['SOSXLTC12AF72A7F54', 6.87],
 ['SOFRQTD12A81C233C0', 5.49],
 ['SOEGIYH12A6D4FC0E3', 4.83],
 ['SOAXGDH12A8C13F8A1', 4.03],
 ['SONYKOW12AB01849C9', 3.5],
 ['SOVDSJC12A58A7A271', 3.28],
 ['SOUFTBI12AB0183F65', 3.05],
 ['SOHTKMO12AB01843B0', 2.97]]

## Step 3: 
For a given user_id, rating, recommend 5 other songs from the list. One way to do this is based on another user who liked the same song liked by this user with rating more than the given rating and recommend the 5 songs based on the matched user's rating. 

In [56]:
combined_rated_triplets = rated_songs.join(hacked_songkey_triplets) \
                                    .map(lambda x: [(x[0],x[1][0]), [x[1][1][0], x[1][1][1]] ]) \
                                    .groupByKey() \
                                    .map(lambda x : (x[0], list(x[1]))) \
                                    .sortBy(lambda x: -x[0][1])

In [57]:
combined_rated_triplets.take(5)

[(('SOBONKR12A58A7A7E0', 10.0),
  [['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', '1'],
   ['c34670d9c1718361feb93068a853cead3c95b76a', '1'],
   ['c5006d9f41f68ccccbf5ee29212b6af494110c5e', '1'],
   ['e4332e11f4df6dd26673bb6b085e9a2bbdc9b8a5', '2'],
   ['baf2fe5885ab93fbbdb7fecc6691788e70afb6c8', '4'],
   ['f6e34f0a68d5ea1344511e33486f956de361db78', '1'],
   ['e326c4b9fe3659ec1dc3af53fd7e0893809dafbc', '25'],
   ['00f7c493ee64884998ea98d9f5bed87bc4a0afcf', '5'],
   ['daa9e7e53ae787ab4f1b5518b695198947d821a2', '1'],
   ['cd4321d8fd42ba44996e7f34c2f6404cf5884696', '1'],
   ['bcb1e6d620cf522390d5c92bae26936928e0b588', '26'],
   ['3d28488a659d1cacc598d17074b233985271c65e', '2'],
   ['eead1f1e3ad91575346f9ce826ddaea19bff80ed', '4'],
   ['07260aa795b86382fbddea37b7b8f6f7d5a1db0a', '53'],
   ['4fbde29be137e7179046c148f9e15db17c3278be', '1'],
   ['ed199f27a41066e37414c3fe9eefb2ae372b8819', '5'],
   ['b6fe63bd640050b922bda5e4ec36d110b878be8a', '40'],
   ['93d158458cbf375218ae7de7fd428bf94cc1491e'

In [101]:
user_id=input("Enter user ID: ")
minrating=input("Enter minimum rating: ")



Enter user ID: 5f3cd0eac9cdeb89958e3927f0f7b230ada8a24f
Enter minimum rating: 5


In [102]:
#Find favorite song of this user
user_songs = triplet_rdd.filter(lambda x: x[0]==user_id)
max_count = user_songs.map(lambda x: int(x[2])).max()
user_songs = triplet_rdd.filter(lambda x: x[0]==user_id and int(x[2])==max_count).map(lambda x: x[1]).collect()

In [103]:
user_songs

['SOAUWYT12A81C206F1']

In [109]:
# Find a different user that has same most played song
def songSearch(row):
    if (row[1] in user_songs and row[0]!=user_id):
        return True
    return False

In [110]:
givenuser_songs = triplet_rdd.filter(songSearch)
sim_user = givenuser_songs.sortBy(lambda x:-int(x[2])).take(1)
sim_user

[['4bacaff23c6edf0cdec81b2a71e0b01df9341734', 'SOAUWYT12A81C206F1', '275']]

In [111]:
similar_user=sim_user[0][0]

In [113]:
# Pick top 5 favorite songs of similar user except our user's favorite song
u_songnames = triplet_rdd.filter(lambda x: (x[0]==similar_user and x[1] not in user_songs)) \
                            .sortBy(lambda x: -int(x[2]))
u_songnames.take(5)

[['4bacaff23c6edf0cdec81b2a71e0b01df9341734', 'SOUNZHU12A8AE47481', '267'],
 ['4bacaff23c6edf0cdec81b2a71e0b01df9341734', 'SOPGCOH12A8C139789', '116'],
 ['4bacaff23c6edf0cdec81b2a71e0b01df9341734', 'SOFRQTD12A81C233C0', '34'],
 ['4bacaff23c6edf0cdec81b2a71e0b01df9341734', 'SOLFXKT12AB017E3E0', '33'],
 ['4bacaff23c6edf0cdec81b2a71e0b01df9341734', 'SOLCKAS12A8C14242A', '32']]

In [116]:
print("Top 5 recommended songs for user: {}".format(user_id))
u_songnames.map(lambda x: x[1]).take(5)

Top 5 recommended songs for user: 5f3cd0eac9cdeb89958e3927f0f7b230ada8a24f


['SOUNZHU12A8AE47481',
 'SOPGCOH12A8C139789',
 'SOFRQTD12A81C233C0',
 'SOLFXKT12AB017E3E0',
 'SOLCKAS12A8C14242A']

## Step 4: 
1. Compute cosine similarity between all pairs of users. 
2. Sort the similarity score and print the top-5 similar users. 
3. If the top-5 user set has an user appearing more than once, ignore that pair and take the next best pair from the sorted list. 
4. For a given user_id, identify the top-5 similar users and hence song recommendations from other user's list. 

In [65]:
numbers = sc.parallelize([[1,7],[8,9],[1,7],[8,8],[1,3],[1,5]])
numbers.collect()

[[1, 7], [8, 9], [1, 7], [8, 8], [1, 3], [1, 5]]

In [56]:
x = numbers.map(lambda x: x[1]).sum()
x


31

In [58]:
from itertools import accumulate

In [69]:
sumx = numbers.groupByKey().mapValues(cumsum)
# numbers.groupByKey().mapValues(cumsum).flatMapValues(lambda x:x).collect()

summ = sumx.map(lambda x: [x[0],x[1][-1]])
summ.collect()

[[8, 17], [1, 22]]